# Realtime Enemy Detector in Call of Duty

First we need to import the base overlay from Pynq and set up our HDMI I/O.

In [1]:
# based off of https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/notebooks/video/opencv_face_detect_hdmi.ipynb

from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [2]:
from pynq.lib.video import *
mode = VideoMode(2560, 1600, 24, 59)

In [3]:
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [4]:
hdmi_in.tie(hdmi_out)

Next we can import the libraries for image processing, and define a function for drawing shapes on an image.

In [4]:
import cv2
import numpy as np

def draw_rectangle(draw_point):
    cv2.rectangle(clip, draw_point, (draw_point[0] + 80, draw_point[1] + 80), (0, 0, 255), 0)

Next we can set up OpenCV's "Simple Blob Detector" and configure the parameters it will use.

In [12]:
params = cv2.SimpleBlobDetector_Params()

params.minThreshold = 10
params.maxThreshold = 80

params.filterByArea = True
params.minArea = 200
params.maxArea = 300

params.filterByCircularity = False
params.minCircularity = 0.1

params.filterByConvexity = False
params.minConvexity = 0.87

params.filterByInertia = False
params.minInertiaRatio = 0.01

detector = cv2.SimpleBlobDetector_create(params)

Next we define our function for processing the frame, making it easier for the blob detector to find blobs.

In [6]:
def process_image(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    # ret, clip = cv2.threshold(blur, 90, 255, cv2.THRESH_BINARY)
    # clip = cv2.Canny(gray, 60, 120, L2gradient=True) 
    return blur

Finally we run the main runloop to show the processed image on HDMI out.

In [11]:
import time

# numframes = 60
# start = time.time()

# for _ in range(numframes):
#     clip = hdmi_in.readframe()
#     blurred = process_image(clip)
#     keypoints = detector.detect(blurred)
#     clip_with_keypoints = cv2.drawKeypoints(blurred, keypoints, clip, (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# #     cv2.imshow("Keypoints", clip)
#     outframe = hdmi_out.newframe()
#     outframe[:] = clip
#     hdmi_out.writeframe(outframe)

Frames per second:  3.720274773025581


In [15]:
# https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/notebooks/video/opencv_filters_hdmi.ipynb

import cv2
import numpy as np
import time
numframes = 200
grayscale = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
blurred = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, 
                           hdmi_in.mode.width), dtype=np.uint8)

start = time.time()

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_RGB2GRAY,dst=grayscale)
    inframe.freebuffer()
    cv2.GaussianBlur(grayscale,(5,5),0,dst=blurred)
    cv2.Canny(grayscale, 100, 110, edges=result)
    outframe = hdmi_out.newframe()
    cv2.cvtColor(result, cv2.COLOR_GRAY2RGB,dst=outframe)
    hdmi_out.writeframe(outframe)
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Frames per second:  11.822246993712536
